<a href="https://colab.research.google.com/github/cwinkelmann/DeepForest/blob/DeformableDETR/docs/user_guide/examples/nest_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for training a nest detection model

### Install Comet ML

In [1]:
!pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.6/728.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 45.4 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


### Install DeepForest library

In [2]:
!git clone https://github.com/weecology/DeepForest.git

Cloning into 'DeepForest'...
remote: Enumerating objects: 17322, done.
remote: Counting objects: 100% (786/786), done.
remote: Compressing objects: 100% (402/402), done.
remote: Total 17322 (delta 593), reused 402 (delta 381), pack-reused 16536 (from 4)
Receiving objects: 100% (17322/17322), 1.00 GiB | 21.52 MiB/s, done.
Resolving deltas: 100% (11764/11764), done.


In [3]:
%cd DeepForest
!pip install -e .
%cd ..

/content/DeepForest
Obtaining file:///content/DeepForest
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.

In [7]:
import os
import sys

deepforest_path = os.path.abspath("DeepForest/src")
deepforest_path

'/content/DeepForest/src'

In [8]:
if deepforest_path not in sys.path:
    sys.path.insert(0, deepforest_path)

In [9]:
# load the modules
import comet_ml
import os
import time
import numpy as np
import pandas as pd
import torch
from deepforest import main
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess
from tqdm import tqdm
from pytorch_lightning.loggers import CometLogger
import zipfile
import matplotlib.pyplot as plt
import subprocess

### Set up Environment Variables

#### In Google Colab
Use Colab's secret storage to securely store your API key.

1. Locate the `Secrets` tab on the left-hand side panel in your Colab notebook.
2. Add a new secret with the key name as `COMET_API_KEY` and paste your Comet ML API key as the value.

#### Locally
Set an environment variable `COMET_API_KEY` in your operating system.

##### Windows
1. Open Command Prompt and set the environment variable:

    ```bash
    setx COMET_API_KEY "your_comet_api_key"
    ```

2. Restart your terminal or IDE.

##### macOS/Linux
1. Open your terminal and add the following line to your `.bashrc`, `.zshrc`, or `.profile` file:

    ```bash
    export COMET_API_KEY="your_comet_api_key"
    ```

2. Save the file and reload the shell configuration:

    ```bash
    source ~/.bashrc  # or ~/.zshrc, ~/.profile, etc.
    ```

In [10]:
PLATFORM = "colab"  # Platform can be colab or local
environment = {}
if PLATFORM == "colab":
    from google.colab import userdata

    environment["api_key"] = userdata.get("COMET_API_KEY")
else:
    environment["api_key"] = os.getenv("COMET_API_KEY")

In [11]:
api_key = environment["api_key"]

In [12]:
# change the project_name
comet_logger = CometLogger(project_name="temporary2", api_key=api_key)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/karisu/temporary2/8280507be70a4a70b7bdfbb11909412f



### Download the Bird nest dataset

In [13]:
root_folder = "/content" if PLATFORM == "colab" else os.environ.get("ROOT_FOLDER")


def download_dataset(output_filename='Dataset.zip', extract_folder_name="dataset"):
    """
    Download a file from a URL using 'wget', extract its contents to a specified folder,
    and handle platform-specific root folder locations.

    Args:
    - output_filename (str): Name of the downloaded file.
    - extract_folder_name (str): Name of the folder to extract the contents into.

    Raises:
    - FileNotFoundError: If the downloaded zip file does not exist.

    Returns:
    None
    """
    url = 'https://www.dropbox.com/s/iczokehl2c5hcjx/nest_images.zip?dl=0'

    # Download the file using wget
    result = subprocess.run(['wget', '-O', output_filename, url],
                            capture_output=True,
                            text=True)

    # Check if the download was successful
    if result.returncode == 0:
        print('Download complete.')
    else:
        print('Error occurred:', result.stderr)

    # Determine the root folder based on the platform

    # Paths for zip file and extraction folder
    zip_file = os.path.join(root_folder, output_filename)
    extract_folder = os.path.join(root_folder, extract_folder_name)

    # Check if the zip file exists
    if not os.path.exists(zip_file):
        raise FileNotFoundError(f"The zip file {zip_file} does not exist.")

    # Create the extract folder if it doesn't exist
    os.makedirs(extract_folder, exist_ok=True)

    # Open the zip file and extract its contents
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        for file in tqdm(zip_ref.namelist(), desc="Extracting", unit="files"):
            zip_ref.extract(file, extract_folder)

    print(f"Successfully unzipped {zip_file} to {extract_folder}.")
    return extract_folder

In [14]:
extract_folder = download_dataset()

Download complete.


Extracting: 100%|██████████| 365/365 [00:19<00:00, 18.46files/s]

Successfully unzipped /content/Dataset.zip to /content/dataset.


In [15]:
# Check if the annotations file has been extracted from the zip file
annotations = pd.read_csv(os.path.join(extract_folder, "nest_data.csv"))
annotations.head()

,image_path,xmin,ymin,xmax,ymax,label,annotator
0,JetPortNew_03_029_2022_DJI_0332.JPG,5414.798306,943.386925,5506.271111,1032.841833,Nest,NaN
1,JetPortNew_03_029_2022_DJI_0332.JPG,5713.079191,887.063465,5819.797463,977.181001,Nest,NaN
2,JetPortNew_03_029_2022_DJI_0332.JPG,5651.434474,758.513449,5790.632221,865.859338,Nest,NaN
3,JetPortNew_03_029_2022_DJI_0332.JPG,5405.787642,348.183777,5508.558036,444.584273,Nest,NaN
4,JetPortNew_03_029_2022_DJI_0332.JPG,5290.340722,235.490240,5386.772855,338.679503,Nest,NaN


In [16]:
# Gather all the images ending with .JPG
image_names = [file for file in os.listdir(extract_folder) if file.endswith(".JPG")]
image_names = image_names[:5]
len(image_names)

100

In [18]:
# Generate crops of the image which has Region of Interest (ROI)
crop_dir = os.path.join(os.getcwd(), "train_data_folder")
annotation_path = os.path.join(extract_folder, "nest_data.csv")
all_annotations = []
for image in image_names:
    image_path = os.path.join(extract_folder, image)
    annotations = preprocess.split_raster(
        path_to_raster=image_path,
        annotations_file=annotation_path,
        patch_size=400,
        patch_overlap=0.0,
        save_dir=crop_dir,
    )
    all_annotations.append(annotations)
train_annotations = pd.concat(all_annotations, ignore_index=True)

Image shape is 3, assuming this is channels last, converting to channels first


/content/DeepForest/src/deepforest/preprocess.py:195: UserWarning: Input image had non-3 band shape of (3, 4008, 6016), selecting first 3 bands
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


Image shape is 3, assuming this is channels last, converting to channels first


/content/DeepForest/src/deepforest/preprocess.py:195: UserWarning: Input image had non-3 band shape of (3, 4008, 6016), selecting first 3 bands
  warnings.warn(


Image shape is 3, assuming this is channels last, converting to channels first


...

/content/DeepForest/src/deepforest/preprocess.py:195: UserWarning: Input image had non-3 band shape of (3, 4008, 6016), selecting first 3 bands
  warnings.warn(


Image shape is 3, assuming this is channels last, converting to channels first


/content/DeepForest/src/deepforest/preprocess.py:195: UserWarning: Input image had non-3 band shape of (3, 4008, 6016), selecting first 3 bands
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1750: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


Image shape is 3, assuming this is channels last, converting to channels first


/content/DeepForest/src/deepforest/preprocess.py:195: UserWarning: Input image had non-3 band shape of (3, 4008, 6016), selecting first 3 bands
  warnings.warn(


In [19]:
image_paths = train_annotations.image_path.unique()

# split into 70% train, 20% validation and 10% test annotations
temp_paths = np.random.choice(image_paths, int(len(image_paths) * 0.30))
valid_paths = np.random.choice(temp_paths, int(len(image_paths) * 0.20))
test_paths = [path for path in temp_paths if path not in valid_paths]

valid_annotations = train_annotations.loc[train_annotations.image_path.isin(valid_paths)]
test_annotations = train_annotations.loc[train_annotations.image_path.isin(test_paths)]
train_annotations = train_annotations.loc[~train_annotations.image_path.isin(temp_paths)]

In [20]:
# View output
print(train_annotations.head())
print("There are {} training crown annotations".format(train_annotations.shape[0]))
print("There are {} test crown annotations".format(valid_annotations.shape[0]))

# save to file and create the file dir
annotations_file = os.path.join(crop_dir, "train.csv")
validation_file = os.path.join(crop_dir, "valid.csv")
test_file = os.path.join(crop_dir, "test.csv")

# Write window annotations file without a header row, same location as the "base_dir" above.
train_annotations.to_csv(annotations_file, index=False)
valid_annotations.to_csv(validation_file, index=False)
test_annotations.to_csv(test_file, index=False)

                         image_path        xmin        ymin        xmax  \
0  Jerrod_03_21_2022DJI_0091_12.png   56.967526   94.049688  110.856167   
1  Jerrod_03_21_2022DJI_0091_14.png    0.000000  230.893627   41.719752   
2  Jerrod_03_21_2022DJI_0091_23.png  323.960416  324.689506  390.555319   
5  Jerrod_03_21_2022DJI_0091_31.png  328.233390  309.251040  397.635024   
6  Jerrod_03_21_2022DJI_0091_32.png  328.233390  301.251040  397.635024   

         ymax label  annotator  \
0  156.876596  Nest        NaN   
1  302.091269  Nest        NaN   
2  390.895182  Nest        NaN   
5  375.719508  Nest        NaN   
6  367.719508  Nest        NaN   

                                            geometry  
0  POLYGON ((56.968 94.05, 56.968 156.877, 110.85...  
1  POLYGON ((41.72 302.091, 41.72 230.894, 0 230....  
2  POLYGON ((323.96 324.69, 323.96 390.895, 390.5...  
5  POLYGON ((328.233 309.251, 328.233 375.72, 397...  
6  POLYGON ((328.233 301.251, 328.233 367.72, 397...  
There are 565 

In [41]:
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra

GlobalHydra.instance().clear()
config_name = "config"

initialize(version_base=None, config_path="pkg://deepforest.conf")
config = compose(config_name=config_name, overrides=[])

# config["architecture"] = 'retinanet'
config["architecture"] = 'DeformableDetr'

m = main.deepforest(config=config, label_dict={"Nest": 0})

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [42]:
# initialize the model and change the corresponding config file
# m = main.deepforest(label_dict={"Nest": 0})

# move to GPU and use all the GPU resources

m.config["train"]["csv_file"] = annotations_file
m.config["train"]["root_dir"] = os.path.dirname(annotations_file)

# Define the learning scheduler type
m.config["train"]["scheduler"]["type"] = "cosine"
m.config["score_thresh"] = 0.4
m.config["train"]["epochs"] = 10
m.config["validation"]["csv_file"] = validation_file
m.config["validation"]["root_dir"] = os.path.dirname(validation_file)

In [43]:
m.config["train"]["scheduler"]["type"]

'cosine'

In [44]:
# create a pytorch lighting trainer used to training
# Disable the sanity check for validation data
m.create_trainer(logger=comet_logger, num_sanity_val_steps=0)
# load the lastest release model (RetinaNet)
# m.load_model('weecology/deepforest-tree')

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..


In [45]:
# Start the training
start_time = time.time()
m.trainer.fit(m)
print(f"--- Training on GPU: {(time.time() - start_time):.2f} seconds ---")

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                 | Type                  | Params | Mode 
-----------------------------------------------------------------------
0 | model                | DeformableDetrWrapper | 40.0 M | train
1 | iou_metric           | IntersectionOverUnion | 0      | train
2 | mAP_metric           | MeanAveragePrecision  | 0      | train
3 | empty_frame_accuracy | BinaryAccuracy        | 0      | train
-----------------------------------------------------------------------
39.8 M    Trainable params
222 K     Non-trainable params
40.0 M    Total params
160.186   Total estimated model params size (MB)
4         Modules in train mode
425       Modules in eval mode
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


Training: |          | 0/? [00:00<?, ?it/s]

COMET INFO: Uploading 5 metrics, params and output messages


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
# save the prediction result to a prediction folder
save_dir = os.path.join(os.getcwd(), "pred_result_test")
results = m.evaluate(test_file,
                     os.path.dirname(test_file),
                     iou_threshold=0.4,
                     savedir=save_dir)

In [ ]:
results["box_precision"]

In [ ]:
results["box_recall"]

In [ ]:
# save the results to a csv file
results["results"].to_csv("results_test_lr_cosine.csv", index=False)

In [ ]:
# Save the model checkpoint
m.trainer.save_checkpoint(
    os.path.join(root_folder, "checkpoint_epochs_10_cosine_lr_retinanet.pl"))

In [ ]:
torch.save(m.model.state_dict(), os.path.join(root_folder, "weights_cosine_lr"))

In [ ]:
# Load from the saved checkpoint
model = main.deepforest.load_from_checkpoint(
    os.path.join(root_folder, "checkpoint_epochs_10_cosine_lr_retinanet.pl"))

In [ ]:
# Add a path to an image to test the model on
path = ""
predicted_image = model.predict_tile(path=path,
                                      return_plot=True,
                                      patch_size=300,
                                      patch_overlap=0.25)
plt.imshow(predicted_image)
plt.show()